---

> ## Bu, Transformer mimarisinin en temel yapı taşıdır.
## Aşağıya sade ama açıklamalı bir PyTorch örneği yazalım.

---

# 🧠 Self-Attention (Scaled Dot-Product) — Kod Yapısı

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [3]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super().__init__()
        self.embed_dim = embed_dim
        
        # 3 ayrı lineer katman: Query, Key, Value
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        
        self.scale = math.sqrt(embed_dim)

    def forward(self, x):
        # x: [batch_size, seq_len, embed_dim]
        Q = self.query(x)   # [B, L, D]
        K = self.key(x)     # [B, L, D]
        V = self.value(x)   # [B, L, D]
        
        # 1️⃣ Skor hesaplama (benzerlik): Q * K^T
        scores = torch.bmm(Q, K.transpose(1, 2)) / self.scale   # [B, L, L]
        
        # 2️⃣ Softmax ile normalize et (her kelime diğerlerine ne kadar dikkat edecek)
        attn_weights = F.softmax(scores, dim=-1)  # [B, L, L]
        
        # 3️⃣ Ağırlıklı toplam: dikkat değerleri * Value
        output = torch.bmm(attn_weights, V)  # [B, L, D]
        
        return output, attn_weights

# 🔍 Adım Adım Açıklama

| Adım | İşlem              | Matematiksel Gösterim           | Açıklama                                                   |
| ---- | ------------------ | ------------------------------- | ---------------------------------------------------------- |
| 1️⃣  | Lineer dönüşüm     | Q = XW_Q, K = XW_K, V = XW_V    | Girdi (X), 3 farklı uzaya projekte edilir                  |
| 2️⃣  | Benzerlik hesapla  | Attention = softmax(QKᵀ / √d_k) | Her kelimenin diğer kelimelere benzerliğini bulur          |
| 3️⃣  | Ağırlıklı ortalama | Output = Attention * V          | “Hangi kelimelere dikkat edilmesi gerektiğini” uygular     |
| 4️⃣  | Sonuç              | Y = Output                      | Artık her kelime bağlama duyarlı bir şekilde temsil edilir |


---

## Şimdi elimizdeki basit Self-Attention modülünü gerçek Transformer standardına daha yakın, performans ve esneklik odaklı bir yapıya çevirelim.
---
### Birkaç iyileştirme yapabiliriz:

## ⚙️ İyileştirme Alanları

#### Layer Normalization

* Çıkışı normalize ederek gradyanların stabil kalmasını sağlar.

#### Residual Connection (Skip Connection)

* Self-Attention çıktısını girdiye ekleyerek öğrenmeyi kolaylaştırır.

#### Dropout

* Aşırı öğrenmeyi (overfitting) engeller.

#### Opsiyonel Masking

* Decoder’da geleceğe bakmayı engeller, Encoder’da mask kullanımı opsiyoneldir.

#### Sade ve okunabilir yapı

* Kodun hem eğitim hem debugging sırasında anlaşılır olması.

# 🔥 Geliştirilmiş Self-Attention — PyTorch

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [ ]:
class ImprovedSelfAttention(nn.Module):
    def __init__(self, embed_dim: int, dropout: float = 0.1):
        super().__init__()
        self.embed_dim = embed_dim
        
        # Query, Key, Value lineer katmanları
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Layer Normalization
        self.norm = nn.LayerNorm(embed_dim)
        
        # Skor ölçekleme
        self.scale = math.sqrt(embed_dim)
    
    def forward(self, x, mask=None):
        """
        x: [batch_size, seq_len, embed_dim]
        mask: opsiyonel, [batch_size, seq_len, seq_len]
        """
        # 1️⃣ Q, K, V hesapla
        Q = self.query(x)  # [B, L, D]
        K = self.key(x)
        V = self.value(x)
        
        # 2️⃣ Skorları hesapla
        scores = torch.bmm(Q, K.transpose(1,2)) / self.scale  # [B, L, L]
        
        # 3️⃣ Mask varsa uygula
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        # 4️⃣ Softmax ve Dropout
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # 5️⃣ Ağırlıklı toplam
        attn_output = torch.bmm(attn_weights, V)  # [B, L, D]
        
        # 6️⃣ Residual + LayerNorm
        out = self.norm(attn_output + x)
        
        return out, attn_weights

## 🔍 İyileştirmelerin Açıklaması

| Özellik                 | Amaç                                                 |
| ----------------------- | ---------------------------------------------------- |
| **Residual Connection** | Öğrenmeyi kolaylaştırır, gradyan kaybını önler       |
| **Layer Normalization** | Daha stabil ve hızlı eğitim sağlar                   |
| **Dropout**             | Overfitting’i azaltır                                |
| **Masking (opsiyonel)** | Decoder için geleceğe bakmayı engeller               |
| **Sade yapı**           | Multi-Head eklemeden temel mekanizmayı optimize eder |


---

## ⚙️ Son İyileştirmeler

#### LayerNorm ve Residual – Eğitim stabilitesi ve gradyan akışı

#### Dropout – Overfitting’e karşı

#### Optional Masking – Decoder için hazır

#### Sade ve anlaşılır yapı – Multi-Head eklemeden temel mekanizmayı optimize ediyoruz

#### Matris çarpım optimizasyonu – bmm ve transpose ile GPU dostu

#### Dikkat ağırlıklarının geri döndürülmesi – Görselleştirme ve debugging için

# 🔥 Son Hâl: Improved Self-Attention

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class FinalSelfAttention(nn.Module):
    def __init__(self, embed_dim: int, dropout: float = 0.1):
        super().__init__()
        self.embed_dim = embed_dim
        
        # Query, Key, Value lineer katmanları
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Layer Normalization
        self.norm = nn.LayerNorm(embed_dim)
        
        # Skor ölçekleme
        self.scale = math.sqrt(embed_dim)
    
    def forward(self, x, mask=None):
        """
        x: [batch_size, seq_len, embed_dim]
        mask: opsiyonel, [batch_size, seq_len, seq_len]
        """
        # 1️⃣ Q, K, V hesapla
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        
        # 2️⃣ Skorları hesapla ve ölçekle
        scores = torch.bmm(Q, K.transpose(1,2)) / self.scale
        
        # 3️⃣ Mask varsa uygula
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        # 4️⃣ Softmax + Dropout
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # 5️⃣ Ağırlıklı toplam
        attn_output = torch.bmm(attn_weights, V)
        
        # 6️⃣ Residual + LayerNorm
        out = self.norm(attn_output + x)
        
        return out, attn_weights


## 🔍 Bu Son Hâlin Avantajları

| Özellik                 | Amaç                                                      |
| ----------------------- | --------------------------------------------------------- |
| **Residual Connection** | Gradyan kaybını önler, öğrenmeyi kolaylaştırır            |
| **Layer Normalization** | Eğitim stabilitesi ve hızlı öğrenme sağlar                |
| **Dropout**             | Overfitting’e karşı                                       |
| **Optional Masking**    | Decoder için geleceğe bakmayı engeller                    |
| **Sade yapı**           | Multi-Head eklemeden temel Self-Attention’ı optimize eder |
| **Attention Weights**   | Görselleştirme ve debugging için geri döndürülür          |


## 💡 Özet:

* Artık elimizde en stabil ve optimize Self-Attention modülü var.

* Bu modül, tek başına veya bir Encoder/Decoder bloğunun içinde kullanılabilir.

* Multi-Head eklemeden önce temel yapıyı son derece sağlamlaştırdık.

| Özellik                    | Önceki Sade Self-Attention          | Final / Geliştirilmiş Self-Attention           | Avantajı                                                                  |
| -------------------------- | ----------------------------------- | ---------------------------------------------- | ------------------------------------------------------------------------- |
| **Residual Connection**    | Yok                                 | Var (`out = LayerNorm(attn_output + x)`)       | Gradyan kaybını önler, öğrenmeyi kolaylaştırır. Eğitim stabilitesi artar. |
| **Layer Normalization**    | Yok                                 | Var (`nn.LayerNorm`)                           | Çıkışları normalize eder, hızlı ve stabil eğitim sağlar.                  |
| **Dropout**                | Var, sadece attention ağırlıklarına | Var, attention ağırlıkları ve softmax sonrası  | Overfitting’i azaltır, model daha genelleyici olur.                       |
| **Masking**                | Opsiyonel ama temel yapı basit      | Opsiyonel ve kodda entegre                     | Decoder’da geleceğe bakmayı engeller, üretim görevlerinde kritik.         |
| **Kod Düzeni / GPU Dostu** | Basit `bmm` ile çarpım              | Aynı `bmm` ama Residual+LayerNorm ile birleşik | Hem okunabilir hem performanslı                                           |
| **Görselleştirme / Debug** | `attn_weights` döndürülüyor         | `attn_weights` döndürülüyor                    | Aynı, ama artık daha stabil.                                              |
| **Matematiksel Stabilite** | Skorlar softmax öncesi direkt       | Skorlar ölçekleniyor (`/√d`) + softmax         | Büyük embedding boyutlarında softmax overflow riskini azaltır             |
